### How to set up this notebook

In order to use this code, first need to follow the instructions at this [link](https://www.notion.so/Archilyse-fdf94f54e9be42f9a68f1609b9ad0ba9) in the section `21.08.2024`.

In [ ]:
%%capture
# for the moment only for this notebook
!git clone https://github.com/facebookresearch/detectron2.git /ifc_dl/submodules/detectron2
%conda install shapely scikit-image -y

Then run this from the CLI:

```
python -m pip install submodules/detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cpu/torch1.9/index.html
```

and restart the kernel 🫡

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from nb_utils.archilyse_auto_utils import plot_predictions
from nb_utils.utils import EcoDatasetTiled512, rmtree
from pdf2image import convert_from_path
from PIL import Image
from tqdm import tqdm

BASE_FOLDER = Path("/Users/simone.bonato/Desktop/ecolution/IFC_DL")

archylise_path = BASE_FOLDER / "submodules/ArchilyseAuto"
sys.path.append(str(archylise_path))

from predictors.predictors.floorplans import FloorplanPredictor
from predictors.predictors.icons import IconPredictor

# from predictors.predictors.roi import RoiPredictor
from predictors.predictors.spaces import SpacePredictor
from predictors.predictors.walls import WallPredictor

### Loading the models

In [ ]:
%%capture
resources_path = archylise_path / "resources"

wall_predictor = WallPredictor(str(resources_path / "walls_model_latest.pth"))
space_predictor = SpacePredictor(str(resources_path / "spaces_model_final.pth"))

icon_pred_version = 2  # 1 or 2

version_str = "" if icon_pred_version == 1 else "_2"
icon_weights_path = str(resources_path / ("icons_model_final" + version_str)) + ".pth"
icon_predictor = IconPredictor(icon_pred_version, icon_weights_path)

### Load an image

In [ ]:
image_path = BASE_FOLDER / "data/cubicasa5k/high_quality/33/F1_scaled.png"
image = Image.open(image_path).convert("RGB")

# plot the img
plt.figure(figsize=(5, 5))
plt.imshow(image)
plt.axis("off")
plt.show()

image = np.array(image)

### Wall prediction

In [ ]:
# import the FloorplanPredictor

# code to make a prediction with the WallPredictor (with the other models its the same, simply change the predictor you are using)
# you can now define several ROIs ---> List[(x_min, y_min, x_max_y_max)]
# to find the ROIs, either you have to do it manually, or you need to train a model to find them for you. We put the whole image for the moment.
# lastly, choose the pixels_per_meter, this is not mandatory and if you do not set this one the model will use 40 by default
wall_predictions = FloorplanPredictor.predict(
    wall_predictor, image, roi=[(0, 0, image.shape[1], image.shape[0])], pixels_per_meter=35
)

In [ ]:
print(f"The class labels are: {set(wall_predictions[0])}")
plot_predictions(image, wall_predictions, "Wall predictions")

### Space prediction

In [ ]:
space_predictions = space_predictor.predict(image)

In [ ]:
print(f"The class labels are: {set(space_predictions[0])}")
plot_predictions(image, space_predictions, "Space predictions")

### Icons prediction

In [ ]:
icon_predictions = icon_predictor.predict(image)

In [ ]:
print(f"The class labels are: {set(icon_predictions[0])}")
plot_predictions(image, icon_predictions, "Icons predictions")

## Predictions on Ecolution data

The images are of the floorplans, since the models used them.

In [ ]:
ecolution_data_path = BASE_FOLDER / "data/ecolution"


floorplan1_path = (
    ecolution_data_path
    / "single-family-house/baeumliweg_0_5325_leibstadt/raw/floorplan/0.pdf"
)

floorplan1 = convert_from_path(floorplan1_path, dpi=50)[0]
floorplan1_array = np.array(floorplan1)

In [ ]:
# get all the predictions
wall_predictions = wall_predictor.predict(floorplan1_array)
space_predictions = space_predictor.predict(floorplan1_array)
icon_predictions = icon_predictor.predict(floorplan1_array)

# plot all the predictions
plot_predictions(floorplan1_array, wall_predictions, "Wall predictions")
plot_predictions(floorplan1_array, space_predictions, "Space predictions")
plot_predictions(floorplan1_array, icon_predictions, "Icons predictions")

In [ ]:
%%capture
%pip install fpdf

In [ ]:
building_prototype_path = BASE_FOLDER / "data/building_prototypes"
cropped_paths = list(building_prototype_path.glob("**/**/*crop*"))
cropped_512 = EcoDatasetTiled512(cropped_paths)

In [ ]:
# preds on the prototype images
from pathlib import Path

from fpdf import FPDF

# create the pdf
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=5)
page_width = pdf.w - 5

figsize = (13, 5)

# create a tmp folder to store the plots
tmp_folder = Path("/ifc_dl/notebooks/tmp")
tmp_folder.mkdir(exist_ok=True)

tqdm_bar = tqdm(cropped_512)
for all_tiles, floorplan_path in tqdm_bar:
    for i, tile in enumerate(all_tiles):
        floorplan_array = (tile.permute(1, 2, 0).numpy() * 255).astype(np.uint8)
        pdf.add_page()
        pdf.set_font("Arial", style="B", size=16)
        pdf.cell(
            200,
            10,
            txt=f"Folder: {str(floorplan_path).encode('latin-1', 'replace').decode('latin-1')}",
            ln=True,
            align="C",
        )
        pdf.set_font("Arial", size=8)
        pdf.cell(
            200,
            10,
            txt=f"Floorplan: {str(floorplan_path).encode('latin-1', 'replace').decode('latin-1')}",
            ln=True,
            align="C",
        )

        # ####### wall preds #######
        wall_predictions = wall_predictor.predict(floorplan_array)
        plot_buf1 = plot_predictions(
            floorplan_array,
            wall_predictions,
            "Wall predictions",
            show=False,
            figsize=figsize,
            buffer=True,
        )
        tmp_path = tmp_folder / f"{floorplan_path}_{i}_wall_predictions.png"
        Image.open(plot_buf1).save(tmp_path)
        pdf.image(str(tmp_path), x=0, y=30, w=page_width)

        # ####### space preds #######
        space_predictions = space_predictor.predict(floorplan_array)
        plot_buf2 = plot_predictions(
            floorplan_array,
            space_predictions,
            "Space predictions",
            show=False,
            figsize=figsize,
            buffer=True,
        )

        tmp_path = tmp_folder / f"{floorplan_path}_{i}_space_predictions.png"
        Image.open(plot_buf2).save(tmp_path)
        pdf.image(str(tmp_path), x=0, y=120, w=page_width)

        # ####### icon preds #######
        icon_predictions = icon_predictor.predict(floorplan_array)
        plot_buf3 = plot_predictions(
            floorplan_array,
            icon_predictions,
            "Icons predictions",
            show=False,
            figsize=figsize,
            buffer=True,
        )

        tmp_path = tmp_folder / f"{floorplan_path}_{i}_icon_predictions.png"
        Image.open(plot_buf3).save(tmp_path)
        pdf.image(str(tmp_path), x=0, y=210, w=page_width)


pdf.output("/ifc_dl/notebooks/floorplans.pdf", "F")
rmtree(tmp_folder)
